## Getting road network graphs

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
from pickle import dump, load
import os
import numpy as np

import sys
sys.path.append("..")

from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import parse_demand, generate_od_xy, add_centroids, get_centroid_grid_coords, auto_configured_centroids
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network, show_demand, show_link_od_flows
from dyntapy.results import get_selected_link_analysis, get_od_flows

# first we download a road network graph using OSMNX, and clean up some of the data. We rename some attributes
# from OpenStreetMap to comply with GMNS (https://github.com/zephyr-data-specs/GMNS) and 
# provide a very rough estimate of capacities.

city = 'Brussels'
g = road_network_from_place(city, buffer_dist_close=5000, buffer_dist_extended=10000)

In [ ]:
# roads in networkx are labelled by their from and to-nodes which come directly from OpenStreetMap
# many of the functions in dyntapy rely on consecutively labelled nodes and links starting from 0
g  = relabel_graph(g)
# after relabelling we can visualize the network
show_network(g, notebook=True)
# networks can be saved as python pickles
HERE = os.path.dirname(os.path.realpath("__file__"))
file_path_network = HERE + os.path.sep +'brussels_road_network'
with open(file_path_network, 'wb') as network_file:
    dump(g, network_file)
    print(f'network saved at f{file_path_network}')

The extracted network has different levels of coarsity, the inner city polygon as defined by OSM retains full detail.
in a 5000 meter buffer around it we retain all roads with values ['trunk, motorway, primary'] for the [highway tag](https://wiki.openstreetmap.org/wiki/Key:highway). For the outer buffer between 5000 -10000 meter from the inner polygon we merely retain ['motorway', 'primary']. This can be configured in the settings. 
we can add auto-configured centroids to the graph, see below. The inner city is filled with centroids on a grid, defined by
inner_city_centroid_spacing
The two buffers act similar as above, they query for the [place](https://wiki.openstreetmap.org/wiki/Key:place) key with ['village', 'city', 'town'] for the inner buffer and ['city', 'town'] for the outer buffers.

In [ ]:
x, y, names, place_tags = auto_configured_centroids(city, buffer_dist_close=5000,
                                                    buffer_dist_extended=10000,
                                                    inner_city_centroid_spacing=600)

connector_type = 'link'
k = 1 # connector pairs (in- and outgoing) per centroid to be generated, snapped to nearest network node
g = add_centroids(g, x, y, k=k, method=connector_type,name=names, place=place_tags)
# adding centroids once again requires relabelling
# centroids are always the first 'nodes' in the network, the other nodes are relabelled
# consecutively in no particular order. Previous ids are stored under 'ext_id' for each node.
# Links are labelled based on their from_nodes, so the centroids outgoing connectors are also the first links.
g = relabel_graph(g)
file_path_network = HERE + os.path.sep +'brussels_road_network_centroids'
with open(file_path_network, 'wb') as network_file:
    dump(g, network_file)
    print(f'network saved at f{file_path_network}')
show_network(g, notebook=True)

******

## Static Traffic assignment

In [ ]:
# file_path_network = HERE + os.path.sep +'brussels_road_network_centroids'
with open(file_path_network, 'rb') as network_file:
    g = load(network_file)

In [ ]:
# generating some random demand

seed = 0
json_demand = generate_od_xy(20, 'Brussels', seed=seed, max_flow=600)
od_graph = parse_demand(json_demand, g)
show_demand(od_graph, notebook=True)


In [ ]:
tolls = np.zeros(g.number_of_edges())
assignment = StaticAssignment(g,od_graph, tolls)
methods = ['dial_b']
for method in methods:
    result = assignment.run(method)
    show_network(g, flows = result.flows, notebook=True, show_nodes=False)
    print(f'{method=} ran successfully')
# all static assignments return a result object that follows the same structure, see
# below
print(result.__dict__.keys())


In [ ]:
print(result.link_costs[1785:1795])

In [ ]:
tolls2 = np.zeros(g.number_of_edges())
tolls2[1791] = 10000
tolls2[1790] = 10000
tolls2[1792] = 10000
assignment2 = StaticAssignment(g,od_graph, tolls2)
methods = ['dial_b']
for method in methods:
    result2 = assignment2.run(method)
    show_network(g, flows = result2.flows, notebook=True, show_nodes=False)
    print(f'{method=} ran successfully')
# all static assignments return a result object that follows the same structure, see
# below
print(result2.__dict__.keys())

In [ ]:
od_flows = get_od_flows(assignment, result)
show_link_od_flows(g, od_flows, notebook=True, show_nodes=False)
# by hovering over the link in question you can inspect the od_flows on every link, they're kept as a list of tuples
# (origin, destination, flow)

In [ ]:
link = 5345 # it could be that, if the graph was changed in OSM this is no longer a loaded link, it that is the case
# simply manually select one of the loaded ones in the figure above
sla_od_flows = get_selected_link_analysis(assignment, od_flows, link)
show_link_od_flows(g, sla_od_flows, highlight_links=[link], notebook=True, show_nodes=False)